In [ ]:
!pip install -U bitsandbytes
!pip install git+https://github.com/huggingface/transformers
!pip install pyngrok
!pip install fastapi uvicorn Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-or4i2n3x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-or4i2n3x
  Resolved https://github.com/huggingface/transformers to commit 37ea04013b34b39c01b51aeaacd8d56f2c62a7eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9969437 sha256=bd82b23c22983cda2aa62901d3db0462dae9558eb1f978fd9c37651707e67997
  Stored in directory: /tmp/pip-ephem-wheel-cache-54jsgm6p/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installa

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig
import torch

# Create a BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Load the model with the new quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    "ProbeMedicalYonseiMAILab/medllama3-v20",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Load the processor
processor = AutoProcessor.from_pretrained("ProbeMedicalYonseiMAILab/medllama3-v20", trust_remote_code=True)

# Move the model to GPU
device = torch.device("cuda")
model = model.eval()
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

In [ ]:
!ngrok config add-authtoken YOUR_AUTH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install python-multipart

In [ ]:
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from PIL import Image
from io import BytesIO
import logging
from pyngrok import ngrok
from typing import Optional, List
from pydantic import BaseModel

app = FastAPI()
logging.basicConfig(level=logging.INFO)


# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

In [ ]:
class DiagnosisRequest(BaseModel):
    query: str

@app.post("/get_diagnosis/")
async def get_diagnosis(query: str = Form(...)):
    try:

        # Process the query text with the model
        inputs = processor(query, return_tensors="pt")
        output = model.generate(**inputs, max_length=1024)

        # Decode the generated output
        decoded_output = processor.decode(output[0], skip_special_tokens=True)

        return {"response": decoded_output}

    except Exception as e:
        return {"error": str(e)}


if __name__ == "__main__":
    import uvicorn
    from uvicorn import Config, Server

    # # Start ngrok tunnel
    public_url = ngrok.connect(8000)
    print("ngrok public URL:", public_url)

    # Start FastAPI server using Uvicorn's Server class
    config = Config(app=app, host="0.0.0.0", port=8000, loop="asyncio")
    server = Server(config=config)
    await server.serve()

INFO:     Started server process [297]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


ngrok public URL: NgrokTunnel: "https://74d3-104-198-97-51.ngrok-free.app" -> "http://localhost:8000"
